In [2]:
import pandas as pd
import numpy as np
import datetime

def readMeanRevertExcel(dateString):

    date = pd.to_datetime(dateString)

    if date.day < 10:
        day = '0'+str(date.day)
    else:
        day = str(date.day)

    if date.month < 10:
        month = '0' + str(date.month)
    else:
        month = str(date.month)

    year = str(date.year)

    nameString = 'MEAN_REVER_OPT_%s_%s_%s.xls' % (day,month,year)

    data = pd.read_excel('C:/Users/D110148/OneDrive - pzem/Modelos y Simulaciones/Mean Reversion Factor vs Aligne/%s' % (nameString), skiprows=5).dropna()
    
    return data.to_records(index=False)

def sortData(aligneInput):
    
    sort_indices = np.lexsort((aligneInput['+1START'],aligneInput['TNUM']))
    
    data = aligneInput[sort_indices]
    
    return data

def filteredByData(sortedData):
    
    firstOcc, firstOccInd = np.unique(sortedData['+1START'], return_index= True)

    datetimes = firstOcc.astype('datetime64[D]')

    weekDay = np.is_busday(datetimes)

    secondOccInd = np.array([firstOccInd[i]+1 for i in range(len(firstOcc)) if weekDay[i] == True])

    allIndices = np.concatenate((firstOccInd,secondOccInd))

    allIndices.sort()

    data = sortedData[allIndices]
    
    return data

def correctedVolData(sAndFData, MRFactor):
    
    sAndFData['+MOD VOL1'] = np.array([[i/float(MRFactor)] for i in sAndFData['+MOD VOL1']]).reshape(sAndFData.shape[0],)
    sAndFData['+MOD VOL2'] = np.array([[i/float(MRFactor)] for i in sAndFData['+MOD VOL2']]).reshape(sAndFData.shape[0],)
    
    return sAndFData
    
    
aligneInput = readMeanRevertExcel('20230301')

sortedData = sortData(aligneInput)

sAndFData = filteredByData(sortedData)

sAndFData = correctedVolData(sAndFData, '0.21')

sAndFData.weight = [0.5 if np.is_busday(i) == True else 1 for i in sAndFData['+1START'].astype('datetime64[D]')]

if all(sAndFData['1MC/2MC'] =='E_PHNL/TENNE2/SLOE/D') == False:
    
    print('Warning: not all the deals left are CSS related')

else:
    
    print('Data Processing succeded: All deals are CSS related')

# sAndFData.dtype.fields
# sAndFData.dtype.names



Data Processing succeded: All deals are CSS related


In [3]:
pd.DataFrame(sAndFData)

,TNUM,+1START,OPT TYPE,1MC/2MC,PC/STRIKE,+MOD CORR,+MOD IR,UNDERLYING1,UNDERLYING2,+MOD VOL1,...,2COMPONENT,+EXPIRY,DEL DATEEND,DEL DATESTART,TT2I CODE,AUD BUSKEY,+MOD TIME,+1QTY,PRC_S1,P&L
0,LVE526,2023-03-03,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.252757,0.026585,120.467021,118.95225,3.658671,...,DAC4TH,2023-03-02,2023-03-31,2023-03-01,17.0,19999972.0,0.002740,3315.16,3.595954,11921.162784
1,LVE526,2023-03-03,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.252757,0.026585,119.065957,118.95225,3.658671,...,DAC4TH,2023-03-02,2023-03-31,2023-03-01,17.0,19999972.0,0.002740,1657.58,2.814095,4664.588073
2,LVE526,2023-03-04,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",-0.081284,0.025308,130,118.052679,4.042421,...,DAC4TH,2023-03-03,2023-03-31,2023-03-01,17.0,19999972.0,0.005479,9945.48,13.059438,129882.383099
3,LVE526,2023-03-05,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",-0.050841,0.024669,110,118.047581,4.296851,...,DAC4TH,2023-03-03,2023-03-31,2023-03-01,17.0,19999972.0,0.005479,9945.48,0.682729,6790.066686
4,LVE526,2023-03-06,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.032419,0.024285,139.205697,118.936791,3.845515,...,DAC4TH,2023-03-03,2023-03-31,2023-03-01,17.0,19999972.0,0.005479,1657.58,20.283293,33621.180566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,MFY165,2025-12-29,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.174556,0.035415,104.900426,123.257234,0.683557,...,DAC4TH,2025-12-26,2025-12-31,2025-12-01,17.0,21793701.0,2.824658,1684.42,4.540510,7648.125055
1770,MFY165,2025-12-30,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.174556,0.035413,111.130183,123.247406,0.683409,...,DAC4TH,2025-12-29,2025-12-31,2025-12-01,17.0,21793701.0,2.832877,1684.42,6.572614,11071.043220
1771,MFY165,2025-12-30,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.174556,0.035413,83.22372,123.247406,0.683409,...,DAC4TH,2025-12-29,2025-12-31,2025-12-01,17.0,21793701.0,2.832877,3368.84,0.750538,2528.442951
1772,MFY165,2025-12-31,Spread,E_PHNL/TENNE2/SLOE/D,"CALL/000,0000",0.174556,0.035411,83.592934,123.237577,0.683262,...,DAC4TH,2025-12-30,2025-12-31,2025-12-01,17.0,21793701.0,2.835616,1684.42,0.782287,1317.700686


In [4]:
#CODE FOR DAILY BASELOAD CSS OPTION VALUATION FUNCTION #THIS IS THE ONE THAT WORKS!!

def DailyCssOptionValuation(date):

    import pandas as pd
    import glob
    import os
    import numpy as np
    import datetime as datetime

    def listPatternFiles(searchDir, pattern):
        files = glob.glob(os.path.join(searchDir, pattern), recursive = True)
        return files

    #Read the hourly power data:

    today = pd.to_datetime(date)
    tYear = today.year
    tMonth = today.month
    tDay = today.day
    
    strMonth = ''
    strDay = ''

    if len(str(tMonth)) < 2:
        strMonth = '0'+str(tMonth)
    else:
        strMonth = str(tMonth)

    if len(str(tDay)) < 2:
        strDay = '0'+str(tDay)
    else:
        strDay = str(tDay)

    pattern = 'Daily_'+str(tYear)+strMonth+strDay+'*HourlyReleveld.csv'    
    searchDir = 'I:/BU Portfolio Analytics/Market Analysis/Power/Models & Tools/Merit Order/PDP/Summary Outputs/'

    HourlyData = pd.read_csv(listPatternFiles(searchDir, pattern)[0], sep = ";")

    #Process the format and transform it to daily data:

    HourlyData['VALUEDATETIME'] = pd.to_datetime(HourlyData['VALUEDATETIME'])
    HourlyData = HourlyData[HourlyData['VALUEDATETIME'] >= pd.Timestamp(today)]
    HourlyData['day'] = [str(i.year) + '-' + str(i.month) + '-' + str(i.day) for i in HourlyData['VALUEDATETIME']]
    HourlyData = HourlyData.groupby('day').agg(np.mean)
    HourlyData.index = pd.to_datetime(HourlyData.index)
    HourlyData.sort_index(inplace = True)

    #Read gas and carbon daily data:

    pattern = 'Daily_'+str(tYear)+strMonth+strDay+'*BASE CASE_Daily.csv'
    gasAndCarbon = pd.read_csv(listPatternFiles(searchDir, pattern)[0], sep = ";")

    #Process the format:

    gasAndCarbon['VALUEDATETIME'] = pd.to_datetime(gasAndCarbon['VALUEDATETIME'])
    gasAndCarbon = gasAndCarbon[gasAndCarbon['VALUEDATETIME'] >= pd.Timestamp(today)]

    #Creating separated dataframes for Carbon and gas from the previous dataframe and setting their index:

    carbonList = ['CARBON_Sim_'+str(i) for i in range(1,501)]
    gasList = ['GAS_Sim_'+str(i) for i in range(1,501)]
    carbonData = gasAndCarbon[carbonList]
    gasData = gasAndCarbon[gasList]
    carbonData.index, gasData.index = (gasAndCarbon['VALUEDATETIME'] for i in range(1,3))

    #Creating CSS database (the efficiency is understated, but this happened to be conservative):

    CssDaily = pd.DataFrame(HourlyData.values - 1.91900818*gasData.values - 0.353084*carbonData.values)
    CssDaily.columns = ['CSS_NL_Sim_'+str(i) for i in range(1,501)]
    CssDaily.index = HourlyData.index
    CssDaily['Year&Month'] = [str(i.year)+ '-' + str(i.month) for i in CssDaily.index]

    #Creating CSS fwd curve:

    CssMonthlyFWD=CssDaily.groupby('Year&Month').agg(np.mean).T.mean().T
    CssMonthlyFWD.index = pd.to_datetime(CssMonthlyFWD.index)
    CssMonthlyFWD.sort_index(inplace = True)

    #Creating CSS fwd option value:

    CssDaily = CssDaily.iloc[:,:-1]
    CssDailyOption = pd.DataFrame(CssDaily.where(CssDaily > 0,0))
    CssDailyOption['Year&Month'] = [str(i.year)+'-'+str(i.month) for i in CssDailyOption.index]
    CssOptMonthly = CssDailyOption.groupby('Year&Month').agg(np.mean)
    CssOptMonthly.index = pd.to_datetime(CssOptMonthly.index)
    CssOptMonthly.sort_index(inplace = True)
    CssOptMonthly = CssOptMonthly.T.mean().T
    
    #Creating Monthly intrinsic:

    CssDailyAvg = CssDaily.T.mean()
    CssDailyIntr = pd.DataFrame(CssDailyAvg.where(CssDailyAvg>0,0))
    CssDailyIntr['Year&Month'] = [str(i.year) + '-' +str(i.month) for i in CssDailyIntr.index]
    CssMonthlyIntr=CssDailyIntr.groupby('Year&Month').agg(np.mean)
    CssMonthlyIntr.index = pd.to_datetime(CssMonthlyIntr.index)
    CssMonthlyIntr.sort_index(inplace=True)
    CssMonthlyIntr = pd.Series(CssMonthlyIntr[0])

    #Creating Final Data Frame:

    data = pd.DataFrame({'Forward_CSS': CssMonthlyFWD,
                  'Forward_Option_Css':CssOptMonthly,
                         'Forward_Intr':CssMonthlyIntr,
                  'Forward_Extrinsic':CssOptMonthly - CssMonthlyIntr})

    #Saving CSV:

    data.to_csv('C:/Users/D110148/OneDrive - pzem/data/CSSExtrinsicMODaily'+str(tYear)+strMonth+strDay+'.csv')
    
    return data

MOData = DailyCssOptionValuation('20230302').to_records()



In [6]:
pd.DataFrame(MOData)

,Year&Month,Forward_CSS,Forward_Option_Css,Forward_Intr,Forward_Extrinsic
0,2023-03-01,-3.485130,4.251027,2.192887,2.058140
1,2023-04-01,-8.578515,2.192914,0.456496,1.736417
2,2023-05-01,-4.369475,3.886534,2.200097,1.686437
3,2023-06-01,-0.520085,6.289048,5.034213,1.254835
4,2023-07-01,-0.384767,6.206943,5.147235,1.059708
5,2023-08-01,-3.340594,5.165756,3.020738,2.145018
6,2023-09-01,13.670140,17.270447,14.658592,2.611855
7,2023-10-01,7.444052,12.209385,9.920749,2.288636
8,2023-11-01,18.275754,21.945156,19.156912,2.788244
9,2023-12-01,3.262139,13.894530,10.421776,3.472754


In [12]:
import sympy as sp
from scipy import stats
import numpy as np
from scipy.optimize import fsolve

def getMeanReversionFactor(f,subset,MOExtrinsic):
    
    v1   = subset['+MOD VOL1']
    v2   = subset['+MOD VOL2']
    s1   = subset['UNDERLYING1'].astype(float)
    s2   = subset['UNDERLYING2'].astype(float)
    p    = subset['+MOD CORR']
    t    = subset['+MOD TIME'].astype(float)
    r    = subset['+MOD IR'].astype(float)
    U = np.array([34.121411565 if i == 'ZB-HTE' else 1 for i in subset['1COMPONENT']])
    w    = subset.weight
    intr = subset['+OPT INTRINSIC']
    v    = (v1**2+v2**2-2*p*v1*v2)**(1/2)
    d1   = (np.log(s1/s2)+(f*v)**2*t/2)/((f*v)*t**(1/2))
    d2   = d1-(f*v)*t**(1/2)
    N1   = stats.norm.cdf(d1)
    N2   = stats.norm.cdf(d2)
    Kirk = np.exp(-r*t)*(s1*N1-s2*N2)*U
    
#----------------------------------------------------
    
    Aligne          = sum(Kirk*w)/sum(w)
    AligneIntrinsic = sum(intr*w)/sum(w)
    AligneExtr      = Aligne - AligneIntrinsic
    
    return AligneExtr - MOExtrinsic

MOExtrinsic = MOData.Forward_Option_Css[:-12] - MOData.Forward_Intr[:-12]

months = list(set(sAndFData['+1START'].astype('datetime64[M]')))
months.sort()

def getMeanReversionFactorArray(months, sAndFData):
    
    factors = []
    
    for i in range(len(months)):
        
        subset = sAndFData[sAndFData['+1START'].astype('datetime64[M]') == months[i]]
        subset.weight = [0.5 if np.is_busday(i) == True else 1 for i in subset['+1START'].astype('datetime64[D]')]
        
        solution = fsolve(getMeanReversionFactor, 1, args=(subset,MOExtrinsic[i]))
        
        factors.append(solution[0])
        
    return factors

getMeanReversionFactorArray(months, sAndFData)  
        

[0.4027874408501483,
 0.3990081464254388,
 0.25172543609468595,
 0.20378383236122655,
 0.1585960663076454,
 0.3002613598740893,
 0.25897570525670965,
 0.18539388790217917,
 0.18250016967705993,
 0.19331180871920736,
 0.21269367031356037,
 0.1701483772523724,
 0.18558220850647095,
 0.26122076579941883,
 0.2567226472248787,
 0.29265107591726613,
 0.24087945949281053,
 0.2342197588605457,
 0.2692383720272029,
 0.26838642477101254,
 0.2297230903671397,
 0.25248971778150836,
 0.18973045728223137,
 0.22214486364147046,
 0.23945380391494636,
 0.2579589894984603,
 0.21640770382443053,
 0.24125285602814173,
 0.2629528872806576,
 0.27619261845180804,
 0.2529598659316179,
 0.16645855932942966,
 0.11576980193087066,
 0.13894004054702164]

In [11]:
months

[numpy.datetime64('2023-03'),
 numpy.datetime64('2023-04'),
 numpy.datetime64('2023-05'),
 numpy.datetime64('2023-06'),
 numpy.datetime64('2023-07'),
 numpy.datetime64('2023-08'),
 numpy.datetime64('2023-09'),
 numpy.datetime64('2023-10'),
 numpy.datetime64('2023-11'),
 numpy.datetime64('2023-12'),
 numpy.datetime64('2024-01'),
 numpy.datetime64('2024-02'),
 numpy.datetime64('2024-03'),
 numpy.datetime64('2024-04'),
 numpy.datetime64('2024-05'),
 numpy.datetime64('2024-06'),
 numpy.datetime64('2024-07'),
 numpy.datetime64('2024-08'),
 numpy.datetime64('2024-09'),
 numpy.datetime64('2024-10'),
 numpy.datetime64('2024-11'),
 numpy.datetime64('2024-12'),
 numpy.datetime64('2025-01'),
 numpy.datetime64('2025-02'),
 numpy.datetime64('2025-03'),
 numpy.datetime64('2025-04'),
 numpy.datetime64('2025-05'),
 numpy.datetime64('2025-06'),
 numpy.datetime64('2025-07'),
 numpy.datetime64('2025-08'),
 numpy.datetime64('2025-09'),
 numpy.datetime64('2025-10'),
 numpy.datetime64('2025-11'),
 numpy.dat